<a href="https://colab.research.google.com/github/TedwardErker/LearnGoogleEarthEngine_TensorFlow_Colab/blob/master/UTC_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```



# Urban Trees via Google Earth Engine and Tensorflow on Google Collab

## Notebook goals


This notebook has several goals.

Further technical skills:
- learn how to use Google Colab
- learn how to use Python's Google Earth Engine API
- learn how to train a convolutional neural network with Tensorflow

Further mission:
- Map urban tree canopy change in two wisconsin counties, Milwaukee and Marathon
- Map urban tree canopy cover in Madison, WI
- Stretch: maps for all cities in the lower 48



### some links to resources
- [Google Earth Engine Documentation](https://developers.google.com/earth-engine)
- [Google Earth Engine API collab setup](https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb#scrollTo=LAZiVi13zTE7)


#Plan

- Set up google earth engine to this collab
- Upload training data for marathon county to google earth engine
- Build simple classifier using just one year of NAIP, assess results
- Upload the lidar derived height raster to google earth engine
- Add the height layer as a feature
- Build simple classifier and asses results
- Upload the Marathon Water layers to google earth engine
- Add water layers as a features
- Build simple classifier and assess results


# Set up
follow [Google Earth Engine API colab setup](https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb#scrollTo=LAZiVi13zTE7)


In [0]:
import ee

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

test the API


In [6]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729
